In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data.csv', )
df.drop(df.columns[0], axis=1, inplace=True)
df.head()

,client_id,order_id,item_name,quantity,price,date
0,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,Cosmonaut,1,16.0,2022-01-24
1,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,Krombacher,1,8.0,2022-01-24
2,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,Ahi Tuna,1,23.0,2022-01-24
3,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,Little Gem Salad,1,14.0,2022-01-24
4,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,Fire Pie,1,20.0,2022-01-24


In [5]:
df.rename(columns = {'item_name':'Left Hand Side'}, inplace=True)

In [6]:
df1 = df.groupby(df['Left Hand Side']).mean()

C:\Users\sanik\AppData\Local\Temp\ipykernel_10288\1843619590.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df1 = df.groupby(df['Left Hand Side']).mean()


In [7]:
df1.drop(['quantity'], axis=1, inplace=True)

In [8]:
df2 = pd.read_csv('Model.csv')
df2.drop(df2.columns[0], axis=1, inplace=True)
df2.head()

,Left Hand Side,Right Hand Side
0,*Beefeter,Etta Old Fashioned
1,*Gran Centenario,Etta Rita
2,*Playpen,Etta Old Fashioned
3,Acorn Squash,Brussels Sprouts
4,Acorn Squash,Burrata


In [9]:
df0 = pd.merge(df2, df1, on='Left Hand Side', how='left')

In [10]:
df0.to_csv('menu.csv')

In [11]:
!pip install --upgrade openai

In [ ]:
import csv
import openai

# Set up your OpenAI API credentials
openai.api_key = API_KEY

# Define your OpenAI GPT-3 model and its parameters
model_engine = "text-davinci-003"
model_name = "EatMeets"

# Read the CSV file
menu_items = []
cart=[]
flag=0
with open("menu.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        menu_items.append(row)

# Define a function to find a menu item by name
def find_menu_item(input_text, menu_items):
    for item in menu_items:
        if item['Left Hand Side'].strip().lower() in input_text.strip().lower():
            cart[item['Left Hand Side']]=item['price']
            return item
        elif "cart" in input_text.strip().lower():
            return("cart")
    return "I'm sorry, I don't understand."


response_text = f"Sure, Here is everything you Ordered, {cart}! Anything else you'd like to order?"


cart= {}
flag=0
# Define a function to generate a response to the user's input
def generate_response(input_text, menu_items, cart, flag):
    input_text = input_text.strip().lower()
    menu_item = find_menu_item(input_text, menu_items)
    response_text = menu_item
    if(response_text != "I'm sorry, I don't understand." 
        and response_text !="cart" and flag==0 and menu_item['Right Hand Side'] not in cart):
        flag=1
        response_text = f"Sure, I suggest you try our {menu_item['Right Hand Side']}! Anything else you'd like to order?"
        print(f"{model_name}: {response_text}")
    elif(response_text == "cart"):
        tc = 0
        order = ''
        for i in cart:
            order = order + ' '+i
            tc+=float(cart[i])
        tc = round(tc, 2)
        response_text = f"Sure, Here is everything you Ordered, {order}! Your total cost is ${tc} Anything else you'd like to order?"
        print(f"{model_name}: {response_text}")
    else:
#         response_text = f"I'm sorry, but I don't understand what you are trying to communicate. Could you please rephrase or provide more context so I can better assist you??"
#         print(f"{model_name}: {response_text}")
        response = openai.Completion.create(
            engine=model_engine,
            prompt=f"input: {input_text}\n",
            max_tokens=50,
            n=1,
            stop=None,
            temperature=0.5
        )
        response_text = response.choices[0].text.strip()
        print(f"{model_name}: {response_text}")

# Set up the initial cart


# Train the OpenAI model using some sample inputs and outputs
formatted_training_data = """
EatMeets: Hello! How can I help you today?

input: I'd like to order a burger please.
EatMeets: Sure thing, one burger added to your cart!

input: Can you show me my Cart?
EatMeets: Sure, Here is everything you Ordered, 1 Burger

input: Can I get a pizza with pepperoni and mushrooms?
EatMeets: Absolutely, our pizza with pepperoni and mushrooms is a customer favorite!We also recommend Our seafood paella is one of our most popular dishes. 

input: Can you show me my Cart?
EatMeets: Sure, Here is everything you Ordered, 1 Burger, 1 Pizza, 1 Mushroom

input: I'll have the paella then.
EatMeets: Excellent choice, I think you'll really enjoy it!

input: Can you show me my Cart?
EatMeets: Sure, Here is everything you Ordered, 1 Burger, 1 Pizza, 1 Mushroom, 1 Paella

"""
response = openai.Completion.create(
    engine=model_engine,
    prompt=formatted_training_data,
    max_tokens=1024,
    n=1,
    stop=None,
    frequency_penalty=0,
    presence_penalty=0,
)

# Save the trained model
model_id = response["model"]

# Test the trained model by generating a response to a prompt
flag=0
while True:
    flag=0
    prompt = input("You: ")
    response_text = generate_response(prompt, menu_items, cart, flag)